# GJ Project: Training the Siamese Network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import distance
import torch.nn as nn
import torchvision.datasets as dsets
from torch.autograd import Variable
import gjnn.model
import gjnn.loss


# Dataset Loading 
dataset = pd.read_csv("~/ds_short.csv", sep=None, engine='python',  dtype={'user_id': "category"})

dataset.head()



,ifp_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,expertise,user_id,value_a,value_b,value_c,days_from_start,a,b,c,distance
0,1001-0,0.175113,0.0,0.676763,0.143534,0.0,0.0,2,600,0.1,0.9,0.0,0,0,1,0,0.141421
1,1001-0,0.175113,0.0,0.676763,0.143534,0.0,0.0,2,2986,0.1,0.9,0.0,0,0,1,0,0.141421
2,1001-0,0.175113,0.0,0.676763,0.143534,0.0,0.0,2,600,0.1,0.9,0.0,0,0,1,0,0.141421
3,1001-0,0.175113,0.0,0.676763,0.143534,0.0,0.0,2,3469,0.2,0.8,0.0,0,0,1,0,0.282843
4,1001-0,0.175113,0.0,0.676763,0.143534,0.0,0.0,2,600,0.1,0.9,0.0,0,0,1,0,0.141421


In [13]:
#dataset = dataset.apply(pd.to_numeric)


# The current split is 95% of data is used for training and 5% for validation of the model
train=dataset.sample(frac=0.95,random_state=200)
test=dataset.drop(train.index)



batch_size = 128
n_iters = 300000
num_epochs = n_iters / (len(train) / batch_size)
num_epochs = int(num_epochs)
print("The number of epochs is: " + str(num_epochs))


train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)
print("After data loading")
# Setting other neural network hyperparameters

hidden_layer_size = 20
siamese_layer_size = 20
output_layer_size = 2
num_features = len(dataset.columns) - 1 
lr = 0.01
momentum = 0.9
num_epoch = 5


model = gjnn.model.SiameseNetwork(num_features, siamese_layer_size, hidden_layer_size, output_layer_size)
print("Model correctly initialized...")


criterion = gjnn.loss.DistanceLoss()
print("Distance Loss Correctly Initialized...")

#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
print("Optimizer Instantiated...")


iter = 0
for epoch in range(num_epochs):
    print("Epoch " + str(epoch))
    for i, features in enumerate(train_loader):
        print("i: " + str(i))
        print("features: " + str(features))
        print("labels: " + str(labels))
        features = Variable(features.view(-1, num_features))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(features)
        
        # We have to compute the distances of a and b from the outcome
        forecast_user_1 = [0, 0, 0]
        forecast_user_2 = [0, 0, 0]
        loss = criterion(forecast_user_1, forecast_user_2, outputs)
        print("loss has been computed for i: " + str(i))
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        # we want to check the accuracy with test dataset every 500 iterations
        # we can change this number, it is just if it is too small we lose a lot of time
        # checking accuracy while if it is big, we have less answers but takes less time for the algorithm
        if iter % 500 == 0:
            # calculate accuracy
            correct = 0
            total = 0
            
            # iterate through test dataset
            for features, labels in test_loader:
                features = Variable(features.view(-1, num_features))
                
                outputs = model(features)
                # get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            print("Iteration: {}. Loss: {}. Accuracy: {}".format(iter, loss.data[0], accuracy))

The number of epochs is: 40
After data loading
Model correctly initialized...
Distance Loss Correctly Initialized...
Optimizer Instantiated...
Epoch 0


KeyError: 639763